In [7]:
import pandas as pd
import json

data = [json.loads(line) for line in 
        open('/home/left/Desktop/dataMining/set2/ask2/yelp_academic_dataset_business.json', 'r')]

In [8]:
data = [x for x in data if x['city']=='Toronto']


In [9]:
data[0]

{'business_id': 'EosRKXIGeSWFYWwpkbhNnA',
 'name': 'Xtreme Couture',
 'address': '700 Kipling Avenue Etobicoke',
 'city': 'Toronto',
 'state': 'ON',
 'postal_code': 'M8Z 5G3',
 'latitude': 43.6245394916,
 'longitude': -79.5291079302,
 'stars': 3.0,
 'review_count': 16,
 'is_open': 1,
 'attributes': {'GoodForKids': 'True',
  'ByAppointmentOnly': 'True',
  'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False}",
  'BikeParking': 'False'},
 'categories': 'Martial Arts, Gyms, Fitness & Instruction, Active Life',
 'hours': {'Monday': '5:30-23:0',
  'Tuesday': '5:30-23:0',
  'Wednesday': '5:30-23:0',
  'Thursday': '5:30-23:0',
  'Friday': '5:30-23:0',
  'Saturday': '8:0-19:0',
  'Sunday': '8:0-17:0'}}

In [6]:
with open('/home/left/Desktop/dataMining/set2/ask2/yelp_academic_dataset_review.json') as myfile:
    head = [next(myfile) for x in range(1)]
print(head)

['{"review_id":"xQY8N_XvtGbearJ5X4QryQ","user_id":"OwjRMXRC0KyPrIlcjaXeFQ","business_id":"-MhfebM0QIsKt87iDN-FNw","stars":2.0,"useful":5,"funny":0,"cool":0,"text":"As someone who has worked with many museums, I was eager to visit this gallery on my most recent trip to Las Vegas. When I saw they would be showing infamous eggs of the House of Faberge from the Virginia Museum of Fine Arts (VMFA), I knew I had to go!\\n\\nTucked away near the gelateria and the garden, the Gallery is pretty much hidden from view. It\'s what real estate agents would call \\"cozy\\" or \\"charming\\" - basically any euphemism for small.\\n\\nThat being said, you can still see wonderful art at a gallery of any size, so why the two *s you ask? Let me tell you:\\n\\n* pricing for this, while relatively inexpensive for a Las Vegas attraction, is completely over the top. For the space and the amount of art you can fit in there, it is a bit much.\\n* it\'s not kid friendly at all. Seriously, don\'t bring them.\\n* 

# Βημα 2

In [39]:
import pandas as pd 
import math
table_R = pd.read_csv("pruned_data.csv")
print(table_R)

        TZQSUDDcA4ek5gBd6BzcjA  qUWqjjjfpB2-4P3He5rsKw  4.0
0       TZQSUDDcA4ek5gBd6BzcjA  6n_MDeYxU1ihB38be9TkVA  4.0
1       TZQSUDDcA4ek5gBd6BzcjA  jo4KmAqlZ7vxjHIP7IIkAw  5.0
2       TZQSUDDcA4ek5gBd6BzcjA  dsAcgF6qtZy2m6d_yWCrGQ  4.0
3       TZQSUDDcA4ek5gBd6BzcjA  siaRCT2-PkyeXUVKrywcTg  4.0
4       TZQSUDDcA4ek5gBd6BzcjA  SGP1jf6k7spXkgwBlhiUVw  5.0
...                        ...                     ...  ...
197106  3L3CI2Om_2SE3T6hxrJKPQ  j_hB9Gt3VMJAbA2JyvyFjw  4.0
197107  3L3CI2Om_2SE3T6hxrJKPQ  WKcVegcUjAld1S737a_QHw  4.0
197108  3L3CI2Om_2SE3T6hxrJKPQ  1FL3oE2mqq_EFAYPd1TWUg  5.0
197109  3L3CI2Om_2SE3T6hxrJKPQ  hlrZHM4D48XiQtXh6cRg_w  1.0
197110  3L3CI2Om_2SE3T6hxrJKPQ  _cVCzKQGt23KKW1M07Yvkw  5.0

[197111 rows x 3 columns]


In [40]:
print("Sample data should have {} rows \n".format(math.ceil(len(table_R)*0.05)))
sample_data = table_R.sample(frac=0.05)
print(len(sample_data))

Sample data should have 9856 rows 

9856


In [41]:
print("After we remove the samples R should have {}".format(orig_table_R_len - len(sample_data)))
table_R = table_R.drop(sample_data.index)
print(len(table_R))

After we remove the samples R should have 187255
187255
